In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE


In [3]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()

Loading dataset MNIST from MNIST
Loading train data... train data of shape (60000, 784) loaded
Loading test data...  test  data of shape (10000, 784) loaded
Data Loaded. Num of features = 784 Num of Classes = 10

In [4]:
param = dict()
param["nFeatures"] = nFeatures

In [5]:
hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]

Generated vanilla HD basis of shape... 

(2500, 784)
Encoding time: 0.11573004722595215 
Dumping basis into file: base_4911.pkl 



4911


In [6]:
train_encoded = HDE.encodeData(traindata, basis)
HDE.saveEncoded(train_encoded, bid, "train")

test_encoded = HDE.encodeData(testdata, basis)
HDE.saveEncoded(test_encoded, bid, "test")

Encoding data of shape (60000, 784)


Time spent: 789 sec
Dumping data into encoded_4911_train.pkl 
Encoding data of shape (10000, 784)


Time spent: 131 sec
Dumping data into encoded_4911_test.pkl 


'encoded_4911_test.pkl'

array([[ 0.26277517, -0.18152338,  0.63876035, ...,  0.00886959,
         0.99789719,  0.29831021],
       [ 0.37551954,  0.59761904,  0.08200804, ..., -0.95374388,
         0.08759336,  0.75179986],
       [-0.13925424,  0.73467132,  0.72827248, ..., -0.41635107,
         0.85380628,  0.16229848],
       ...,
       [ 0.39246484,  0.57711764,  0.42839617, ..., -0.88375828,
         0.96032739, -0.10287583],
       [-0.55426046,  0.35933053, -0.09409063, ..., -0.8938067 ,
         0.52565331, -0.11344214],
       [ 0.35126811, -0.25962545,  0.88135312, ...,  0.00240632,
         0.91300799, -0.21511451]])